In [44]:
import itertools
import numpy as np
import pandas as pd
import pickle
import datetime
import statsmodels.api as sm
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn import linear_model
import time
import scipy.stats as sps

* Files needed to extract from BigQuery weekly: (1) csv files of order records upto the cutoff date for top10 items (2) daily active users csv file upto the cutoff date (3) sellout feature upto week n-2 (endInventory table)

* Cutoff date for week n: Sunday of week n-2

In [45]:
def unifDate(d):
    '''
    This function takes a date string in month/date/Year pattern,
    and returns a date string in Year-month-day pattern.
    '''
    return datetime.datetime.strptime(d, "%m/%d/%Y").strftime("%Y-%m-%d")

def datemod(data):
    '''
    This function takes the whole dataset and converts all order dates into 
    adjusted business days stored in a list. 
    '''
    Datemod = []
    for i in range(len(data)):
        hour, minute, sec = data.orderTime.iloc[i].split(':')
        hour = int(hour)
        if i == 0 or hour > 2:
            Datemod.append(data.orderDate.iloc[i])
        else:
            d = data.orderDate.iloc[i]
            d = datetime.datetime.strptime(d, "%Y-%m-%d")
            d = d- datetime.timedelta(days=1)
            d = d.strftime("%Y-%m-%d")
            Datemod.append(d)
    return Datemod

def week_n(data,fall_18=0,spring_19=0,spring_18=0):
    '''
    This function takes the whole dataset, fall/spring boolean and returns a new column of week_number
    in a fall/spring semester stored in a list.
    '''
    week_num = []
    if fall_18==1:
        start = datetime.datetime.strptime('2018-09-02', "%Y-%m-%d")
    if spring_19==1:
        start = datetime.datetime.strptime('2019-01-13', "%Y-%m-%d")
    if spring_18 == 1:
        start = datetime.datetime.strptime('2018-01-14', "%Y-%m-%d")
    for i in range(len(data)):
        d = data['Datemod'].iloc[i]
        d = datetime.datetime.strptime(d, "%Y-%m-%d")
        week_num.append(int(np.ceil((d - start)/ datetime.timedelta(days=7))))
    return week_num

def week_n_au(data,fall_18=0,spring_19=0,spring_18=0):
    '''
    THis function takes a data frame about daily active user amounts, fall/spring boolean
    and returns the corresponding week_number in the semester that will refer to 
    this value stored in a list.
    '''
    week_num = []
    if fall_18==1:
        start = datetime.datetime.strptime('2018-08-19', "%Y-%m-%d")
    if spring_19==1:
        start = datetime.datetime.strptime('2018-12-30', "%Y-%m-%d")
    if spring_18==1:
        start = datetime.datetime.strptime('2017-12-31', "%Y-%m-%d")
    for i in range(len(data)):
        d = data['formattedDate'].iloc[i]
        d = datetime.datetime.strptime(d, "%Y-%m-%d")
        week_num.append(int(np.ceil((d - start)/ datetime.timedelta(days=7))))
    return week_num

In [37]:
def clean_item_19spring(filename,enddate,pred_enddate,dau_filename):
    """
    This funciton takes (1) csv filename of a top10 item (2) the cutoff date of order records (3) the last date of our prediction 
    (4) csv filename of daily active users upto the cutoff date as arguments, and returns an organized dataframe 
    including all features for a top10 item.
    """
    data= pd.read_csv(filename,dtype = str,index_col=0)
    item_name = data.itemName.iloc[0]
    data = data[['orderDate', 'orderTime','itemQuantity', 'itemPricePerUnit', 'itemPriceTotal','mealPlanBoolean','circuitName', 'dayOfWeek','discountPercent', 
            'discountPerUnit','discountTotal', 'discountType','deliveryDate']]
    data.orderDate = data.orderDate.apply(unifDate)
    data.mealPlanBoolean = data.mealPlanBoolean.apply(lambda x: 1 if x=='true' else 0)
    data.itemQuantity = data.itemQuantity.apply(int)
    data.itemPricePerUnit = data.itemPricePerUnit.apply(float)

    data['Datemod'] = pd.Series(datemod(data),index=data.index)
    data = data[[ 'itemQuantity', 'itemPricePerUnit',  'circuitName', 'dayOfWeek', 'Datemod']]
    data.set_index('Datemod', drop = False,inplace=True)
    data = data.loc[:enddate]
    daily_sales = data.groupby(by = 'Datemod').itemQuantity.sum()

    new_index = pd.date_range(start='2019-01-14', end=pred_enddate, freq='D')
    new_index=new_index.astype(str)

    data.drop_duplicates(subset=['Datemod'], keep='first', inplace = True)
    data = data.reset_index(drop=True)
    data.index=data.Datemod
    data = data.reindex(new_index)
    data.Datemod = data.index

    #Monday: 0 Sunday:6
    data['dayOfWeek'] = data.Datemod.apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").weekday())
    data = data[data['Datemod']>='2019-01-14']

    # Add cycle_number feature
    data['cycle_n'] = np.where(((data['dayOfWeek'] == 0)|(data['dayOfWeek'] == 1)|(data['dayOfWeek'] == 2)),1,
             np.where(((data['dayOfWeek'] == 3)|(data['dayOfWeek'] == 4)),2,3))
    # Add actual 19spring week number
    data['week_n'] = pd.Series(week_n(data,spring_19=1),index=data.index)

    df_sales=pd.DataFrame(daily_sales)
    df_sales = df_sales.rename(columns={'itemQuantity':'sales'})

    data = pd.merge(data,df_sales,how='left',left_index=True,right_index=True)
    data1=data[data.columns.difference(['circuitName','itemQuantity'])]
    
    # Adjust daily sales according to discount situation
    perc = pd.read_pickle("C:/Users/Baoyp/Documents/2019 Spring/GU project/premodel/premodelDF.pkl")
    discount = pd.read_pickle("C:/Users/Baoyp/Documents/2019 Spring/GU project/discount.p")
    discount['discount'] = 1
    discount_item=discount[discount['item_name']==item_name][['Date','discount']]

    data1 = data1.merge(discount_item,how='left',left_on='Datemod',right_on='Date')
    del data1['Date']
    data1.fillna(0,inplace=True)
    perc_item=perc[perc['name']==item_name][['weekday','factor']]
    data1 = data1.merge(perc_item,how='left',left_on='dayOfWeek',right_on='weekday')
    del data1['weekday']
    weekly_sales=data1.groupby('week_n').sales.sum()
    data1.discount = data1.discount.apply(int)
    idx = data1.index[data1['discount']]==1
    if len(data1.loc[idx]) == 0:
        pass
    elif len(data1.loc[idx]) == 1:
        data1.loc[idx,'sales']=data1.loc[idx]['factor']*weekly_sales.loc[data1.loc[idx]['week_n']]
    elif len(data1.loc[idx]) >1:
        data1.loc[idx,'sales']=data1.loc[idx]['factor'].values*weekly_sales.loc[data1.loc[idx]['week_n']].values

    # Get sales data
    sales = data1.groupby(by = ['week_n', 'cycle_n']).sales.sum()
    sales = sales.reset_index(level=['week_n', 'cycle_n'])

    df1 = pd.read_csv('Academic_Calander_Spring_2019.csv')
    df1 = df1.iloc[:110]
    df1.deliveryDate = df1.deliveryDate.apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%y").strftime("%Y-%m-%d"))
    df1 = df1.rename(columns = {'deliveryDate':'Datemod','class':'classes'})
    data1 = pd.merge(data1,df1, on = 'Datemod', how='left')

    # Get exam days feature
    df_exam = data1.groupby(['week_n','cycle_n']).exam.sum()
    df_exam = pd.DataFrame(df_exam,columns=['exam'])
    df_exam.exam = np.where(df_exam.exam>0,1,0)

    # Get class days feature
    df_class = data1.groupby(['week_n','cycle_n']).classes.sum()
    df_class = pd.DataFrame(df_class,columns=['classes'])
    # Get sports days feature
    df_sport = data1.groupby(['week_n','cycle_n']).sports.sum()
    df_sport = pd.DataFrame(df_sport,columns=['sports'])
    df_sport.sports = np.where(df_sport.sports>0,1,0)

    data1 = data1[['cycle_n', 'itemPricePerUnit','week_n']]
    # Get price feature, it hasn't been used yet
    price = data.groupby(by = ['week_n', 'cycle_n']).itemPricePerUnit.min()
    price = price.reset_index(level=['week_n', 'cycle_n'])

    final = pd.merge(data1, sales, how = 'left', on = ['week_n', 'cycle_n'])
    final.drop_duplicates(subset=['week_n', 'cycle_n'], keep='first', inplace = True)
    final = final.reset_index(drop=True)
    final = final[['week_n','cycle_n','itemPricePerUnit','sales']]
    final = final.rename(columns = {'itemPricePerUnit':'price'})
    final.sales = np.where((final['cycle_n']==1),final['sales']/3,final['sales']/2)

    final = final[~(final['week_n']==8)]
    final = final[~(final['week_n']==9)]

    # Get 2-week lag feature
    final['last_2_week_sales'] = final['sales']
    final['last_2_week_sales']=final['last_2_week_sales'].shift(6)

    final = pd.merge(final,df_class,on = ['week_n','cycle_n'], how='left')
    final = pd.merge(final,df_exam,on = ['week_n','cycle_n'], how='left')
    final = pd.merge(final,df_sport,on = ['week_n','cycle_n'], how='left')

    main_data = pd.get_dummies(final, prefix='Cycle_', columns=['cycle_n'])

    # Get weekly active users feature
    au = pd.read_csv(dau_filename)
    au = au.iloc[9:]
    au = au.reset_index(drop=True)
    au = au[au['formattedDate']<=enddate]
    au['week_n'] = pd.Series(week_n_au(au,spring_19=1),index=au.index)

    if final.iloc[-1]['week_n']==10:
        for i in range(42,49):
            au.loc[i,'week_n']=10
            
    if final.iloc[-1]['week_n']==11:
        for i in range(42,49):
            au.loc[i,'week_n']=11

    au = au.rename(columns={'f0_':'n_users'})
    df_au = au.groupby('week_n').n_users.sum()
    df_au = df_au.reset_index(drop=False)


    main_data = pd.merge(main_data,df_au,how='left',on='week_n')

    # Get average sales over last three cycles feature
    df_weekly = main_data.groupby(by=['week_n']).sales.mean()

    avg_over_3_cycles = df_weekly.shift(2)
    avg_over_3_cycles=avg_over_3_cycles.reset_index()
    avg_over_3_cycles=avg_over_3_cycles.rename(columns={'sales':'avg_over_last_3_cycles'})

    main_data = pd.merge(main_data,avg_over_3_cycles[['week_n','avg_over_last_3_cycles']], on = 'week_n',how = 'left')
    main_data = main_data.drop('price',axis=1)
    main_data['item_name'] = item_name
    main_data = main_data.dropna()

    # Add sellout feature
    sellout = pd.read_pickle('C:/Users/Baoyp/Documents/2019 Spring/GU project/week10(3.18-3.24)/sellout_df_spring19.pkl')
    sellout = sellout[(sellout['week_n']!=0)&(sellout['productName']==item_name)]
    sellout['ref_week_n'] = np.where(sellout['week_n']!=7,sellout['week_n']+2,sellout['week_n']+4)
    del sellout['week_n']
    sellout = sellout.rename(columns={'ref_week_n':'week_n'})
    del sellout['sellout']
    sellout = sellout.rename(columns={'productName':'item_name'})

    main_data['cycle_n'] = np.where(main_data['Cycle__1']==1,1, np.where(main_data['Cycle__2']==1,2,3))
    main_data = pd.merge(main_data,sellout,how='left')
    main_data= main_data.rename(columns = {'75_percent':'sellout'})
    main_data['sellout*2_week_lag']=main_data['last_2_week_sales']*main_data['sellout']
    main_data = main_data[['week_n', 'sales', 'last_2_week_sales', 'classes', 'exam', 'sports',
           'Cycle__1', 'Cycle__2', 'Cycle__3', 'n_users', 'avg_over_last_3_cycles','sellout','sellout*2_week_lag','cycle_n','item_name']]
    
    return main_data

In [38]:
import warnings
warnings.filterwarnings('ignore')

In [39]:
# filename='C:/Users/Baoyp/Documents/2019 Spring/GU project/by2.17/top1.csv'
# enddate = '2019-02-17'
# dau_filename = 'DAU_for_week7.csv'

enddate = '2019-03-03'
pred_enddate = '2019-03-24'
dau_filename = 'C:/Users/Baoyp/Documents/2019 Spring/GU project/week10(3.18-3.24)/DAU_for_week10.csv'
filenames = [f'C:/Users/Baoyp/Documents/2019 Spring/GU project/week10(3.18-3.24)/top{i}.csv' for i in range(1,11)]
dfs = [clean_item_19spring(filename,enddate,pred_enddate,dau_filename) for filename in filenames]
main_data = pd.concat(dfs)
main_data1 = main_data.sort_values(by=['week_n'])
main_data1.reset_index(inplace=True,drop=True)
main_data1.reset_index(inplace=True,drop=True)

Fit into the final model

In [41]:
### CHANGED!
final_model = pickle.load(open('final_log_model_stats.p',"rb" ))
df = main_data1.copy() #pickle.load(open('final_dataset_19spring_week8.p',"rb" ))
name_list = df.item_name.unique()

In [42]:
pred_week_n=10
test_data =df[df['week_n']==pred_week_n]
test_data = test_data.sort_values(by=['item_name','cycle_n'],axis=0)
del test_data['sales']

In [17]:
test_data = test_data [(test_data['item_name']== 'Mac and Cheese')| (test_data['item_name']== 'Mexican Burrito Bowl')| 
            (test_data['item_name']== 'Penne alla Vodka')|(test_data['item_name']== 'Pad Thai with Chicken') |
            (test_data['item_name']== 'Roasted Chicken Plate')]

In [19]:
#test_data['sales']=np.log(test_data['sales'])
test_data['n_users'] = np.log(test_data['n_users'])
test_data['avg_over_last_3_cycles']=np.log(test_data['avg_over_last_3_cycles'])
test_data['last_2_week_sales'] = np.log(test_data['last_2_week_sales'])
test_data['sellout*2_week_lag'] = test_data['sellout']*test_data['last_2_week_sales']
test_data['week_n'] = np.log(test_data['week_n'])

In [21]:
#  The order of variables here muse be aligned with the order of input variables in the final model
### CHANGED
X_test = test_data[['Cycle__2', 'Cycle__3', 'avg_over_last_3_cycles', 'exam', 'last_2_week_sales', 'n_users', 'sellout' , 'week_n']]
X_test = sm.add_constant(X_test, has_constant='add')
Y_pred =  final_model.predict(X_test)
####### test_data['Y_pred'] = np.exp(Y_pred)
test_data['Y_pred'] = Y_pred
########
#test_data['Final_pred'] = np.where((test_data.cycle_n==1),test_data['Y_pred']*3,test_data['Y_pred']*2)
pred_output = test_data[['item_name','Y_pred','week_n','cycle_n']]
pred_output = pred_output.reset_index(drop=True)

In [29]:
### CHANGED
def simu_new(row,alpha):
    '''
    This function takes output from main model as lambda, and return order quantity Q that keeps wastage level at 0.3.
    '''
    Ys = np.random.normal(row['mean'],row['std'],1000)
    Xs = np.exp(Ys)
    Xs = sorted(Xs)
    return Xs[int(1000*alpha)-1]

In [30]:
###CHANGED
df_loc = pred_output['Y_pred']
summary = final_model.get_prediction(X_test).summary_frame(alpha=0.05)
df_se=(summary['obs_ci_upper']-summary['obs_ci_lower'])/(2*1.96)
df_se = df_se.reset_index(drop=True)
#df_se = np.where(df_se.index%3==0,df_se*3,df_se*2)
df_dist = pd.DataFrame({'mean':df_loc, 'std':df_se})

In [15]:
###CHANGED
Q_GU = 990
error=list()
alpha_array = np.linspace(0,1,10000,endpoint=False)
for alpha in alpha_array:
    Qs = df_dist.apply(lambda x: simu_new(x,alpha),axis=1)
    Qs = np.where(Qs.index%3==0,Qs*3,Qs*2)
    Q = np.sum(Qs)
    error.append(Q-Q_GU)
alpha_opt = alpha_array[np.argmin(np.absolute(error))]
alpha_opt

0.9066000000000001

In [16]:
np.array(error)[(alpha_array==alpha_opt)]

array([0.01104908])

In [17]:
Qs = df_dist.apply(lambda x: simu_new(x,alpha_opt),axis=1)
Qs = np.where(Qs.index%3==0,Qs*3,Qs*2)
pred_output['simu_result'] = Qs

In [18]:
pred_output.to_csv('week10_mainmodel_output.csv')

In [ ]:
### UP TO HERE